In [1]:
from python_on_whales import DockerClient, docker
import uuid

docker_clients = []
for i in range(1):
    worker_id = uuid.uuid4().hex
    docker_client = DockerClient(
        compose_project_name=worker_id,
        compose_files=["client-compose.yml"],
    )
    docker_clients.append(docker_client)



In [3]:
for docker_client in docker_clients:
    docker_client.compose.up(
        detach=True,
        build=True,
    )

#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 121B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 121B done
#2 WARN: LegacyKeyValueFormat: "ENV key=value" should be used instead of legacy "ENV key value" format (line 7)
#2 DONE 0.0s

#3 [app-account internal] load metadata for docker.io/library/golang:latest
#3 DONE 0.0s

#4 [app-product internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [app-account 1/4] FROM docker.io/library/golang:latest
#5 DONE 0.0s

#6 [app-account internal] load .dockerignore
#6 transferring context: 2B 0.0s done
#6 DONE 0.0s

#7 [app-product internal] load build context
#7 transferring context: 5.08kB 0.0s done
#7 DONE 0.0s

#8 [app-product 2/4] WORKDIR /workdir
#8 CACHED

#9 [app-account internal] load build context
#9 transferring context: 335B 0.0s done
#9 DON

KeyboardInterrupt: 

In [3]:
docker.image.list()

[python_on_whales.Image(id='sha256:b4201f2066453', tags=['tiangolo/uvicorn-gunicorn-fastapi:latest']),
 python_on_whales.Image(id='sha256:0db8fd8d8896a', tags=[]),
 python_on_whales.Image(id='sha256:5f6fa28313b81', tags=[]),
 python_on_whales.Image(id='sha256:43283e931cec8', tags=[]),
 python_on_whales.Image(id='sha256:a61b645b609b7', tags=['golang:latest']),
 python_on_whales.Image(id='sha256:cf8d6ecc3574a', tags=['muazhari/autocode-example-app-account:latest']),
 python_on_whales.Image(id='sha256:1c0fd857deda9', tags=['muazhari/autocode-example-app-product:latest']),
 python_on_whales.Image(id='sha256:28e8ac7aafcf5', tags=[])]

In [5]:
for docker_client in docker_clients:
    docker_client.compose.down()

 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Stopping
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Stopping
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Stopped
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Removing
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Stopped
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Removing
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Removed
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Removed
 Network 577f83d86ebd46d59dcdd5c466efb96f_default  Removing
 Network 577f83d86ebd46d59dcdd5c466efb96f_default  Removed
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Stopping
 Container 9bae1087874f45d885f2d86ae6fe903f-app-account-1  Stopping
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Stopped
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Removing
 Container 9bae1087874f45d885f2d86ae6fe903f-app-account-1  Stopped
 Co

In [15]:
list(docker_clients[0].compose.ps()[0].network_settings.networks.values())[0].ip_address

[NetworkInspectResult(ipam_config=None, links=None, aliases=['f4425bc7e176405e89ca8af2954caa46-app-account-1', 'app-account'], network_id='00785bea842e5b3820f4f95902ec47281ceb44bba67df60dfdc6bdbfd20763b0', endpoint_id='', gateway='', ip_address='', ip_prefix_length=0, ipv6_gateway='', global_ipv6_address='', global_ipv6_prefix_length=0, mac_address='', driver_options=None)]

In [17]:
for docker_client in docker_clients:
    for container in docker_client.compose.ps():
        A_WORKER_ID = docker_client.client_config.compose_project_name
        B_WORKER_ID = [env for env in container.config.env if "WORKER_ID" in env][0].split("=")[1]
        print(A_WORKER_ID, B_WORKER_ID, A_WORKER_ID == B_WORKER_ID, container.name)

f4425bc7e176405e89ca8af2954caa46 f4425bc7e176405e89ca8af2954caa46 True f4425bc7e176405e89ca8af2954caa46-app-account-1
f4425bc7e176405e89ca8af2954caa46 f4425bc7e176405e89ca8af2954caa46 True f4425bc7e176405e89ca8af2954caa46-app-product-1


In [20]:
container.name.removeprefix("f4425bc7e176405e89ca8af2954caa46-")

'app-product-1'

In [5]:
from httpx import Client

http_client = Client(
    base_url="http://172.18.0.3:8000",
)
http_client.get("/").json()

{'worker_id': 'd7d6146b1c2c4fb8ac4934277a567520', 'app_name': 'app-1'}

In [1]:
import autocode

optimization = autocode.Optimization(
    num_cpus=2,
    server_host="0.0.0.0",
    server_port=10000,
    dashboard_port=10001,
)

2024-08-04 00:37:26,361	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at http://172.25.4.192:8265 


Available resources:  {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'node:172.25.4.192': 1.0, 'CPU': 2.0, 'memory': 9101891175.0, 'object_store_memory': 4550945587.0, 'GPU': 1.0}


 Container 939c8e5e58d64ea7a9bc86fbb7cbacaf-app-account-1  Stopping
 Container 939c8e5e58d64ea7a9bc86fbb7cbacaf-app-product-1  Stopping
 Container 939c8e5e58d64ea7a9bc86fbb7cbacaf-app-account-1  Stopped
 Container 939c8e5e58d64ea7a9bc86fbb7cbacaf-app-account-1  Removing
 Container 939c8e5e58d64ea7a9bc86fbb7cbacaf-app-product-1  Stopped
 Container 939c8e5e58d64ea7a9bc86fbb7cbacaf-app-product-1  Removing
 Container 939c8e5e58d64ea7a9bc86fbb7cbacaf-app-product-1  Removed
 Container 939c8e5e58d64ea7a9bc86fbb7cbacaf-app-account-1  Removed
 Network 939c8e5e58d64ea7a9bc86fbb7cbacaf_default  Removing
 Network 939c8e5e58d64ea7a9bc86fbb7cbacaf_default  Removed
 Container 906c929a33e64aa7af012952a8b4ca5e-app-product-1  Stopping
 Container 906c929a33e64aa7af012952a8b4ca5e-app-account-1  Stopping
 Container 906c929a33e64aa7af012952a8b4ca5e-app-product-1  Stopped
 Container 906c929a33e64aa7af012952a8b4ca5e-app-product-1  Removing
 Container 906c929a33e64aa7af012952a8b4ca5e-app-account-1  Stopped
 Co

Server is healthy.


In [2]:
docker_clients = optimization.deploy(
    compose_files=["./client-compose.yml"],
)

#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 101B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 101B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load metadata for docker.io/library/golang:latest
#3 DONE 0.0s

#4 [app-account internal] load .dockerignore
#4 transferring context: 2B 0.0s done
#4 DONE 0.0s

#5 [app-product internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-account 1/4] FROM docker.io/library/golang:latest
#6 DONE 0.0s

#7 [app-product internal] load build context
#7 transferring context: 335B 0.0s done
#7 DONE 0.0s

#8 [app-account internal] load build context
#8 transferring context: 335B 0.0s done
#8 DONE 0.0s

#9 [app-product 3/4] COPY . .
#9 CACHED

#10 [app-product 4/4] RUN go mod tidy
#10 CACHED

#11 [app-account 2/4] WORKDIR /workdir
#11 CACHED

#12

 Network beefe62c70404696b98cb0ff23f33af1_default  Creating
 Network beefe62c70404696b98cb0ff23f33af1_default  Created
 Container beefe62c70404696b98cb0ff23f33af1-app-account-1  Creating
 Container beefe62c70404696b98cb0ff23f33af1-app-product-1  Creating
 Container beefe62c70404696b98cb0ff23f33af1-app-account-1  Created
 Container beefe62c70404696b98cb0ff23f33af1-app-product-1  Created
 Container beefe62c70404696b98cb0ff23f33af1-app-account-1  Starting
 Container beefe62c70404696b98cb0ff23f33af1-app-product-1  Starting
 Container beefe62c70404696b98cb0ff23f33af1-app-account-1  Started
 Container beefe62c70404696b98cb0ff23f33af1-app-product-1  Started
gio: http://localhost:10001: Operation not supported


#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 101B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 101B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load metadata for docker.io/library/golang:latest
#3 DONE 0.0s

#4 [app-product internal] load .dockerignore
#4 transferring context: 2B 0.0s done
#4 DONE 0.0s

#5 [app-account internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-account 1/4] FROM docker.io/library/golang:latest
#6 DONE 0.0s

#7 [app-account internal] load build context
#7 transferring context: 335B 0.0s done
#7 DONE 0.0s

#8 [app-product internal] load build context
#8 transferring context: 335B 0.0s done
#8 DONE 0.0s

#9 [app-product 3/4] COPY . .
#9 CACHED

#10 [app-product 4/4] RUN go mod tidy
#10 CACHED

#11 [app-product 2/4] WORKDIR /workdir
#11 CACHED

#12

 Network 9177604a897b46c8bb17a3f6462b7980_default  Creating
 Network 9177604a897b46c8bb17a3f6462b7980_default  Created
 Container 9177604a897b46c8bb17a3f6462b7980-app-account-1  Creating
 Container 9177604a897b46c8bb17a3f6462b7980-app-product-1  Creating
 Container 9177604a897b46c8bb17a3f6462b7980-app-account-1  Created
 Container 9177604a897b46c8bb17a3f6462b7980-app-product-1  Created
 Container 9177604a897b46c8bb17a3f6462b7980-app-account-1  Starting
 Container 9177604a897b46c8bb17a3f6462b7980-app-product-1  Starting
 Container 9177604a897b46c8bb17a3f6462b7980-app-account-1  Started
 Container 9177604a897b46c8bb17a3f6462b7980-app-product-1  Started


In [3]:
docker_clients[0].container.list()[0].network_settings.networks

{'9177604a897b46c8bb17a3f6462b7980_default': NetworkInspectResult(ipam_config=None, links=None, aliases=['9177604a897b46c8bb17a3f6462b7980-app-product-1', 'app-product'], network_id='5711b357babdc25aaf14a60a997f54a8243c697cd3832890bfcf32b082f1f5c7', endpoint_id='ea6a2d4c25bbb8f681dc9a1ad2678ece740e48e3e925f9055d222923055b7515', gateway='192.168.160.1', ip_address='192.168.160.2', ip_prefix_length=20, ipv6_gateway='', global_ipv6_address='', global_ipv6_prefix_length=0, mac_address='02:42:c0:a8:a0:02', driver_options=None)}

In [4]:
from autocode import OptimizationObjective
from typing import Dict, Any, List
from autocode import OptimizationEvaluateRunResponse

objectives: List[OptimizationObjective] = [
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),

    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
]


def evaluator(input: List[OptimizationEvaluateRunResponse]) -> Dict[str, Any]:
    output: Dict[str, Any] = {
        "F": input[0].objectives + input[1].objectives,
    }

    return output


In [5]:
result = optimization.run(
    objectives=objectives,
    num_inequality_constraints=0,
    num_equality_constraints=0,
    evaluator=evaluator
)

RayTaskError(RemoteProtocolError): [36mray::__call__()[39m (pid=7908, ip=172.25.4.192)
  File "/home/muazhari/miniconda3/lib/python3.11/site-packages/httpx/_transports/default.py", line 373, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/muazhari/miniconda3/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 216, in handle_async_request
    raise exc from None
  File "/home/muazhari/miniconda3/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 196, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/muazhari/miniconda3/lib/python3.11/site-packages/httpcore/_async/connection.py", line 101, in handle_async_request
    return await self._connection.handle_async_request(request)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/muaz